# __Analisis de Evasion de Clientes__

## Librerias

In [270]:
import pandas as pd
import numpy as np

In [271]:
import numexpr
print(numexpr.__version__)

2.11.0


## Extraccion de datos

### Importar Datos

In [272]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

df = pd.read_json(url)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Como vemos, hay varias variables que contienen diccionarios en sus valores. Por tanto, debemos expandirlas

In [273]:
# expandir cada variable
df_customer = pd.json_normalize(df['customer'])
df_phone = pd.json_normalize(df['phone'])
df_internet = pd.json_normalize(df['internet'])
df_account = pd.json_normalize(df['account'])

In [274]:
# seleccionar variables no expandidas de df
df = df[['customerID', 'Churn']]

In [275]:
# unir todos los dfs en uno
df_exp = pd.concat([df, df_customer, df_phone, df_internet, df_account], axis=1)
df_exp.head(5)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


### Contexto de los datos

Se tratan de datos de los clientes de una empresa cuya informacion de cada variable se interpreta como:

- _customerID_: número de identificación único de cada cliente
- _Churn_: si el cliente dejó o no la empresa
- _gender_: género (masculino y femenino)
- _SeniorCitizen_: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- _Partner_: si el cliente tiene o no una pareja
- _Dependents_: si el cliente tiene o no dependientes
- _tenure_: meses de contrato del cliente
- _PhoneService_: suscripción al servicio telefónico
- _MultipleLines_: suscripción a más de una línea telefónica
- _InternetService_: suscripción a un proveedor de internet
- _OnlineSecurity_: suscripción adicional de seguridad en línea
- _OnlineBackup_: suscripción adicional de respaldo en línea
- _DeviceProtection_: suscripción adicional de protección del dispositivo
- _TechSupport_: suscripción adicional de soporte técnico, menor tiempo de espera
- _StreamingTV_: suscripción de televisión por cable
- _StreamingMovies_: suscripción de streaming de películas
- _Contract_: tipo de contrato
- _PaperlessBilling_: si el cliente prefiere recibir la factura en línea
- _PaymentMethod_: forma de pago
- _Charges.Monthly_: total de todos los servicios del cliente por mes
- _Charges.Total_: total gastado por el cliente

### Tipo de datos

In [276]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


> El conjunto de datos esta conformado por 7267 registros y 21 variables.

> Todas las variables tienen 7267 registros lo que indica la ausencia de valores faltantes.

> Las variables 'SeniorCitizen' y 'tenure' son de tipo entero, la variable 'Charges.Monthly' es de tipo float y el resto de variables son de tipo object.

> La variable 'Charges.Total' y 'Churn' en la practica son de tipo numerico. Sin embargo, Pandas las detecta de tipo object lo que sugiere un problema con la consistencia de los datos.

## Consistencia de los datos

### Registros duplicados

In [277]:
# verificar registros duplicados
df_exp.duplicated().sum()

0

> No hay registros duplicados en el dataframe.

### Tratar valores vacios

Verificamos los valores de 'Churn' y 'Charges.Total'

In [278]:
print('Charges.Total: ',df_exp['Charges.Total'].unique())
print('Churn: ',df_exp.Churn.unique())

Charges.Total:  ['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']
Churn:  ['No' 'Yes' '']


> Los valores de _Churn_ y _Charges.Total_ presentan valores vacios por lo que procedemos a tratarlos

__Churn__

In [279]:
# registros con valores vacios en _Churn_
df_exp[df_exp['Churn'] == ''].shape

(224, 21)

> Hay 224 registros con valores vacios en _Churn_ lo que representa el 3% de los registros totales. Como la variable objetivo es _Churn_, sin esta informacion tales registros no aportan al analisis por tanto los eliminamos.

In [280]:
# elimina registros con Churn vacios
df_exp = df_exp[df_exp['Churn'] != '']
df_exp.shape

(7043, 21)

In [281]:
# verificar que Churn este libre de valores vacios
print(df_exp.Churn.unique())

['No' 'Yes']


__Charges.Total__

In [282]:
# registros vacios en Charges.Total
df_exp[df_exp['Charges.Total'] == ' ']

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,


> Hay 11 registros con valores vacios en Charges.Total lo que representa el 0.1% de los registros totales. Al visualizarlos, nos damos cuenta que corresponden a clientes con contratos largos pero que no tienen ni el primer mes cumplido por lo que no aportan informacion relevante al estudio. Los eliminamos

In [283]:
# elimina registros con Charges.Total vacios
df_exp = df_exp[df_exp['Charges.Total'] != ' ']
df_exp.shape

(7032, 21)

In [284]:
# verificar que Churn este libre de valores vacios
print(df_exp['Charges.Total'].unique())

['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']


In [285]:
import numpy as np 

# convertir a tipo entero
df_exp['Charges.Total'] = df_exp['Charges.Total'].astype(np.float64)

# verificar tipo
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   Churn             7032 non-null   object 
 2   gender            7032 non-null   object 
 3   SeniorCitizen     7032 non-null   int64  
 4   Partner           7032 non-null   object 
 5   Dependents        7032 non-null   object 
 6   tenure            7032 non-null   int64  
 7   PhoneService      7032 non-null   object 
 8   MultipleLines     7032 non-null   object 
 9   InternetService   7032 non-null   object 
 10  OnlineSecurity    7032 non-null   object 
 11  OnlineBackup      7032 non-null   object 
 12  DeviceProtection  7032 non-null   object 
 13  TechSupport       7032 non-null   object 
 14  StreamingTV       7032 non-null   object 
 15  StreamingMovies   7032 non-null   object 
 16  Contract          7032 non-null   object 
 17  

### Columna de cuentas diarias

In [286]:
# crear columna de gasto diario de cada cliente
df_exp['Cuentas_Diarias'] = (df_exp['Charges.Monthly'])/(df_exp['tenure']*30)
df_exp.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,0.242963
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,0.221852
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,0.615833
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,0.251282
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,0.932222


### Estandarizacion de los datos

Vamos a realizar las conversiones de los valores de las variables necesarias para mejorar la interpretacion de los resultados.

__Valores Unicos por variable__

In [287]:
# verificar valores de variables de tipo object
print('gender: ',df_exp.gender.unique())
print('SeniorCitizen: ',df_exp.SeniorCitizen.unique())
print('Partner: ',df_exp.Partner.unique())
print('Dependents: ',df_exp.Dependents.unique())
print('PhoneService: ',df_exp.PhoneService.unique())
print('MultipleLines: ',df_exp.MultipleLines.unique())
print('InternetService: ',df_exp.InternetService.unique())
print('OnlineSecurity: ',df_exp.OnlineSecurity.unique())
print('OnlineBackup: ',df_exp.OnlineBackup.unique())
print('DeviceProtection: ',df_exp.DeviceProtection.unique())
print('TechSupport: ',df_exp.TechSupport.unique())
print('StreamingTV: ',df_exp.StreamingTV.unique())
print('StreamingMovies: ',df_exp.StreamingMovies.unique())
print('Contract: ',df_exp.Contract.unique())
print('PaperlessBilling: ',df_exp.PaperlessBilling.unique())
print('PaymentMethod: ',df_exp.PaymentMethod.unique())
print('Charges.Total: ',df_exp['Charges.Total'].unique())
print('Churn: ',df_exp.Churn.unique())

gender:  ['Female' 'Male']
SeniorCitizen:  [0 1]
Partner:  ['Yes' 'No']


Dependents:  ['Yes' 'No']
PhoneService:  ['Yes' 'No']
MultipleLines:  ['No' 'Yes' 'No phone service']
InternetService:  ['DSL' 'Fiber optic' 'No']
OnlineSecurity:  ['No' 'Yes' 'No internet service']
OnlineBackup:  ['Yes' 'No' 'No internet service']
DeviceProtection:  ['No' 'Yes' 'No internet service']
TechSupport:  ['Yes' 'No' 'No internet service']
StreamingTV:  ['Yes' 'No' 'No internet service']
StreamingMovies:  ['No' 'Yes' 'No internet service']
Contract:  ['One year' 'Month-to-month' 'Two year']
PaperlessBilling:  ['Yes' 'No']
PaymentMethod:  ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
Charges.Total:  [ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]
Churn:  ['No' 'Yes']


__Cambiar nombre de las variables__

In [288]:
nombres_nuevos = {
    'Churn': 'cancelacion',
    'gender': 'genero',
    'SeniorCitizen': 'jubilado',
    'Partner': 'en_pareja',
    'Dependents': 'dependientes',
    'tenure': 'meses_contrato',
    'PhoneService': 'servicio_telefonico',
    'MultipleLines': 'multiples_lineas',
    'InternetService': 'servicio_internet',
    'OnlineSecurity': 'seguridad_online',
    'OnlineBackup': 'respaldo_online',
    'DeviceProtection': 'proteccion_dispositivo',
    'TechSupport': 'soporte_efectivo',
    'StreamingTV': 'streaming_TV',
    'StreamingMovies': 'streaming_peliculas',
    'Contract': 'tipo_contrato',
    'PaperlessBilling': 'facturacion_electronica',
    'Charges.Monthly': 'cuenta_mensual',
    'Charges.Total': 'cuentas_total',
    'PaymentMethod': 'metodo_pago',
    'Cuentas_Diarias': 'cuentas_diarias'
}

df_exp.rename(columns=nombres_nuevos, inplace=True)

__customerID__

Vamos a omitir customer_ID ya que son identificadores de los clientes que a nivel general no aporta informacion relevante.

In [289]:
# eliminar customerID
df_exp = df_exp.drop('customerID', axis=1)

__genero__

In [290]:
# renombrar valores de genero
df_exp['genero'] = df_exp['genero'].apply(lambda x: x.replace('Female', 'Femenino').replace('Male', 'Masculino'))
df_exp['genero'].unique()

array(['Femenino', 'Masculino'], dtype=object)

__jubilado__

In [291]:
df_exp['jubilado'] = df_exp['jubilado'].map({0:'No', 1:'Si'})
df_exp['jubilado'].unique()

array(['No', 'Si'], dtype=object)

__cancelacion__

In [292]:
# convertir valores de churn a binario
# 0:No esta con la empresa. 1: Sigue en la empresa
df_exp['cancelacion'] = df_exp['cancelacion'].replace({'Yes':'1', 'No':'0'}).astype(np.int64)
df_exp['cancelacion'].unique()

array([0, 1])

__en_pareja__

In [293]:
df_exp['en_pareja'] = df_exp['en_pareja'].map({'Yes':'Si', 'No':'No'})
df_exp.en_pareja.unique()

array(['Si', 'No'], dtype=object)

__dependientes__

In [294]:
df_exp['dependientes'] = df_exp['dependientes'].replace({'Yes':'Si', 'No':'No'})
print(df_exp.dependientes.unique())

['Si' 'No']


__facturacion_electronica__

In [295]:
df_exp['facturacion_electronica'] = df_exp['facturacion_electronica'].replace({'Yes':'Si', 'No':'No'})
print(df_exp.facturacion_electronica.unique())

['Si' 'No']


__servicio_telefonico__

In [296]:
df_exp['servicio_telefonico'] = df_exp['servicio_telefonico'].replace({'Yes':'Si', 'No':'No'})
print(df_exp.servicio_telefonico.unique())

['Si' 'No']


__multiples_lineas__

In [297]:
# agrupamiento de los valores de servicio_telefonico y multiples_lineas
df_exp.groupby(['servicio_telefonico', 'multiples_lineas']).size()

servicio_telefonico  multiples_lineas
No                   No phone service     680
Si                   No                  3385
                     Yes                 2967
dtype: int64

> Los clientes que tienen servicio telefonico necesariamente tienen al menos 1 linea. Los que no tienen servicio telefonico, no tienen linea telefonica. Por tanto, los datos son consistentes.

__servicio_internet__

In [298]:
df_exp['servicio_internet'] = df_exp['servicio_internet'].replace({'Fiber optic':'Fibra_optica'})
df_exp['servicio_internet'].unique()

array(['DSL', 'Fibra_optica', 'No'], dtype=object)

__seguridad_online__

In [299]:
df_exp['seguridad_online'] = df_exp['seguridad_online'].replace({'Yes': 'Si', 'No internet service': 'Sin internet'})
df_exp.seguridad_online.unique()

array(['No', 'Si', 'Sin internet'], dtype=object)

In [300]:
df_exp.groupby(['servicio_internet', 'seguridad_online']).size()

servicio_internet  seguridad_online
DSL                No                  1240
                   Si                  1176
Fibra_optica       No                  2257
                   Si                   839
No                 Sin internet        1520
dtype: int64

Los clientes que no tienen servicio de internet, no cuentan con seguridad online. Esto indica que los datos son consistentes.

__respaldo_online__

In [301]:
df_exp['respaldo_online'] = df_exp['respaldo_online'].replace({'Yes': 'Si', 'No internet service': 'Sin internet'})
df_exp.respaldo_online.unique()

array(['Si', 'No', 'Sin internet'], dtype=object)

In [302]:
df_exp.groupby(['servicio_internet', 'respaldo_online']).size()

servicio_internet  respaldo_online
DSL                No                 1334
                   Si                 1082
Fibra_optica       No                 1753
                   Si                 1343
No                 Sin internet       1520
dtype: int64

Los clientes que no tienen internet, no cuentan con respaldo online lo que implica que ambas variables son consistentes.

__proteccion_dispositivo, soporte_efectivo, streaming_tv, streaming_peliculas__

__proteccion_dispositivo__

In [303]:
df_exp['proteccion_dispositivo'] = df_exp['proteccion_dispositivo'].replace({'Yes': 'Si', 'No internet service': 'Sin internet'})
df_exp.proteccion_dispositivo.unique()

array(['No', 'Si', 'Sin internet'], dtype=object)

In [304]:
df_exp.groupby(['servicio_internet', 'proteccion_dispositivo']).size()

servicio_internet  proteccion_dispositivo
DSL                No                        1355
                   Si                        1061
Fibra_optica       No                        1739
                   Si                        1357
No                 Sin internet              1520
dtype: int64

Los clientes que no tienen internet, no cuentan con proteccion de dispositivo. Son consistentes los datos.

__soporte_efectivo__

In [305]:
df_exp['soporte_efectivo'] = df_exp['soporte_efectivo'].replace({'Yes': 'Si', 'No internet service': 'Sin internet'})
df_exp.soporte_efectivo.unique()

array(['Si', 'No', 'Sin internet'], dtype=object)

In [306]:
df_exp.groupby(['servicio_internet', 'soporte_efectivo']).size()

servicio_internet  soporte_efectivo
DSL                No                  1242
                   Si                  1174
Fibra_optica       No                  2230
                   Si                   866
No                 Sin internet        1520
dtype: int64

Clientes sin internet, no cuentan con soporte efectivo. Se comprueba la consistencia de los datos.

__streaming_TV__

In [307]:
df_exp['streaming_TV'] = df_exp['streaming_TV'].replace({'Yes': 'Si', 'No internet service': 'Sin internet'})
df_exp.streaming_TV.unique()

array(['Si', 'No', 'Sin internet'], dtype=object)

In [308]:
df_exp.groupby(['servicio_internet', 'soporte_efectivo']).size()

servicio_internet  soporte_efectivo
DSL                No                  1242
                   Si                  1174
Fibra_optica       No                  2230
                   Si                   866
No                 Sin internet        1520
dtype: int64

Clientes sin internet, no cuentan con streaming de TV. Datos consistentes

__streaming_peliculas__

In [309]:


df_exp['streaming_peliculas'] = df_exp['streaming_peliculas'].replace({'Yes': 'Si', 'No internet service': 'Sin internet'})
df_exp.streaming_peliculas.unique()

array(['No', 'Si', 'Sin internet'], dtype=object)

In [310]:
df_exp.groupby(['servicio_internet', 'streaming_peliculas']).size()

servicio_internet  streaming_peliculas
DSL                No                     1436
                   Si                      980
Fibra_optica       No                     1345
                   Si                     1751
No                 Sin internet           1520
dtype: int64

Clientes sin internet, no cuentan con streaming de peliculas

__tipo_contrato__

In [311]:
df_exp['tipo_contrato'] = df_exp['tipo_contrato'].replace(
    {'One year':'Un year',
     'Month-to-month':'mes a mes',
     'Two year':'Dos years'
     }
)

df_exp.tipo_contrato.unique()

array(['Un year', 'mes a mes', 'Dos years'], dtype=object)

__metodo_pago__

In [312]:
df_exp['metodo_pago'] = df_exp['metodo_pago'].replace(
    {'Mailed check':'Cheque por correo',
     'Electronic check': 'Cheque electronico',
     'Credit card (automatic)':'Tarjeta de credito',
     'Bank transfer (automatic)':'Transferencia'
     }
    )

df_exp.metodo_pago.unique()

array(['Cheque por correo', 'Cheque electronico', 'Tarjeta de credito',
       'Transferencia'], dtype=object)

In [313]:
df_exp.head()

,cancelacion,genero,jubilado,en_pareja,dependientes,meses_contrato,servicio_telefonico,multiples_lineas,servicio_internet,seguridad_online,...,proteccion_dispositivo,soporte_efectivo,streaming_TV,streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago,cuenta_mensual,cuentas_total,cuentas_diarias
0,0,Femenino,No,Si,Si,9,Si,No,DSL,No,...,No,Si,Si,No,Un year,Si,Cheque por correo,65.6,593.30,0.242963
1,0,Masculino,No,No,No,9,Si,Yes,DSL,No,...,No,No,No,Si,mes a mes,No,Cheque por correo,59.9,542.40,0.221852
2,1,Masculino,No,No,No,4,Si,No,Fibra_optica,No,...,Si,No,No,No,mes a mes,Si,Cheque electronico,73.9,280.85,0.615833
3,1,Masculino,Si,Si,No,13,Si,No,Fibra_optica,No,...,Si,No,Si,Si,mes a mes,Si,Cheque electronico,98.0,1237.85,0.251282
4,1,Femenino,Si,Si,No,3,Si,No,Fibra_optica,No,...,No,Si,Si,No,mes a mes,Si,Cheque por correo,83.9,267.40,0.932222


In [314]:
df_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cancelacion              7032 non-null   int64  
 1   genero                   7032 non-null   object 
 2   jubilado                 7032 non-null   object 
 3   en_pareja                7032 non-null   object 
 4   dependientes             7032 non-null   object 
 5   meses_contrato           7032 non-null   int64  
 6   servicio_telefonico      7032 non-null   object 
 7   multiples_lineas         7032 non-null   object 
 8   servicio_internet        7032 non-null   object 
 9   seguridad_online         7032 non-null   object 
 10  respaldo_online          7032 non-null   object 
 11  proteccion_dispositivo   7032 non-null   object 
 12  soporte_efectivo         7032 non-null   object 
 13  streaming_TV             7032 non-null   object 
 14  streaming_peliculas      7032

## Carga y Analisis

### Resumen estadistico variables categoricas 

In [315]:
# lista de variables categoricas
var_categoricas = df_exp.select_dtypes(include=['object', 'category']).columns
#  dataframe con el resumen estadistico por variable
df_categoricas = df_exp[var_categoricas].describe()
# trasponer dataframe
df_categoricas = df_categoricas.T

In [316]:
# columna con la proporcion del valor top de cada variable
df_categoricas['proporcion_top'] = df_categoricas['freq']/df_categoricas['count']
df_categoricas

,count,unique,top,freq,proporcion_top
genero,7032,2,Masculino,3549,0.504693
jubilado,7032,2,No,5890,0.8376
en_pareja,7032,2,No,3639,0.517491
dependientes,7032,2,No,4933,0.701507
servicio_telefonico,7032,2,Si,6352,0.903299
multiples_lineas,7032,3,No,3385,0.481371
servicio_internet,7032,3,Fibra_optica,3096,0.440273
seguridad_online,7032,3,No,3497,0.497298
respaldo_online,7032,3,No,3087,0.438993
proteccion_dispositivo,7032,3,No,3094,0.439989


> La proporcion entre hombres y mujeres son similares. (50.4% masculino, 50.6% femenino)

> EL 83.76% de clientes no son adultos mayores.

> La proporcion entre clientes en pareja y solteros son similares

> 70.1% de los clientes no tienen dependientes.

> 90.33% de los clientes cuentan con servicio telefonico.

> 48.1% de los clientes cuentan con una linea teleonica.

> 44.03% de los clientes cuentan con internet de tipo fibra optica.

> 49.7% de los clientes no cuenta con seguridad online.

> Solo el 43.9% de los clientes no tienen respaldo online.

> 44% de los clientes no cuentan con proteccion de dispositivo.

> 49.3% de los clientes no cuentan con soporte tecnico eectivo.

> 40% de los clientes no cuentan con servicio de streaming TV.

> 40% de los clientes no cuentan con servicio de streaming de peliculas.

> 55.1% de los clientes tienen un tipo de contrato mes a mes

> La mayoria de clientes prefieren recibir factura electronica.

> El metodo de pago predilecto por los clientes es Cheque Electronico.

### Resumen estadistico variables numericas

In [317]:
# dataframe con variables numericas
var_numericas = df_exp.select_dtypes(include=['number']).columns
df_exp[var_numericas].describe()

,cancelacion,meses_contrato,cuenta_mensual,cuentas_total,cuentas_diarias
count,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.265785,32.421786,64.798208,2283.300441,0.287336
std,0.441782,24.545260,30.085974,2266.771362,0.545069
min,0.000000,1.000000,18.250000,18.800000,0.008935
25%,0.000000,9.000000,35.587500,401.450000,0.042590
50%,0.000000,29.000000,70.350000,1397.475000,0.071576
75%,1.000000,55.000000,89.862500,3794.737500,0.220374
max,1.000000,72.000000,118.750000,8684.800000,3.415000


> Alrededor del 26.5% de los clientes cancelaron el servicio. 

> Hay contratos tan cortos como 1 mes y tan largos como 72 meses

> El 25% de los clientes tiene contratos de 9 meses o menos y el 75% de 55 meses o menos lo que muestra una alta variabilidad.

> Los pagos mensuales van de 18.25 a 89.86

> La diferencia entre la mediana y la media en los pago mensuales indican una asimetria a la izquierda.

> La diferencia entre la mediana y la media en las cuentas totales evidencia la presencia de outliers positivos

> La alta desviacion estandar en las cuentas totales indican que la distribucion no es normal

> La mediana esta muy por debajo del promedio en las cuentas totales lo que indica que una minoria gasta mucho mas.

> La diferencia entre la mediana y el promedio en cuentas diarias indica que la distribucion es asimetrica a la derecha.

> El 75% de los clientes pagan menos de 0.22 por dia.

### Resumen estadistico de variables categoricas de clientes que cancelan

In [318]:
# lista de variables categoricas
var_categoricas = df_exp.select_dtypes(include=['object', 'category']).columns

# obtener dataframe con variables categoricas mas la variable cancelacion
df_c = df_exp[var_categoricas].copy()
df_c['cancelacion'] = df_exp['cancelacion'].copy()

# seleccionar registros con clientes que cancelaron
df_c_cancelaron = df_c[df_c['cancelacion'] == 1]

# resumen estadistico de las variables categoricas
df_c_cancelaron = df_c_cancelaron.describe(exclude='int')

# trasponer filas por columnas
df_c_cancelaron = df_c_cancelaron.T

# agregar la columna 'proporcion top'
df_c_cancelaron['proporcion_top'] = df_c_cancelaron['freq']/df_c_cancelaron['count']
df_c_cancelaron

,count,unique,top,freq,proporcion_top
genero,1869,2,Femenino,939,0.502408
jubilado,1869,2,No,1393,0.745318
en_pareja,1869,2,No,1200,0.642055
dependientes,1869,2,No,1543,0.825575
servicio_telefonico,1869,2,Si,1699,0.909042
multiples_lineas,1869,3,Yes,850,0.454789
servicio_internet,1869,3,Fibra_optica,1297,0.693954
seguridad_online,1869,3,No,1461,0.781701
respaldo_online,1869,3,No,1233,0.659711
proteccion_dispositivo,1869,3,No,1211,0.64794


> En total, 1869 clientes cancelaron el servicio con la empresa

> La proporcion de hombres y mujeres que cancelaron son similares.

> 64% de los clientes son solteros.

> 82.5% de los clientes no tienen dependientes a su cargo.

> 91% tienen servicio telefonico

> 45% tiene mas de 1 linea telefonica.

> 69% cuenta con servicio de internet a traves de fibra optica

> 78% no cuenta con seguridad en linea

> 65% no posee respaldo en linea

> 64% no cuenta con proteccion de dispositivos.

> 77% no tienen soporte tecnico efectivo.

> 50% de los clientes no acceden al servicio de streaming TV.

> 50% de los clientes no acceden al servicio de streaming de peliculas.

> El tipo de contrato preferido por estos clientes es el mes a mes.

> La mayoria prefieren facturacion electronica

> EL metodo de pago mas utlizado por este grupo es el cheque electronico.

### Resumen estadistico de variables categoricas de clientes que no cancelan

In [319]:
# lista de variables categoricas
var_categoricas = df_exp.select_dtypes(include=['object', 'category']).columns

# obtener dataframe con variables categoricas mas la variable cancelacion
df_c = df_exp[var_categoricas].copy()
df_c['cancelacion'] = df_exp['cancelacion'].copy()

# seleccionar registros con clientes que  no cancelaron
df_c_Nocancelaron = df_c[df_c['cancelacion'] == 0]

# resumen estadistico de las variables categoricas
df_c_Nocancelaron = df_c_Nocancelaron.describe(exclude='int')

# trasponer filas por columnas
df_c_Nocancelaron = df_c_Nocancelaron.T

# agregar la columna 'proporcion top'
df_c_Nocancelaron['proporcion_top'] = df_c_Nocancelaron['freq']/df_c_Nocancelaron['count']
df_c_Nocancelaron

,count,unique,top,freq,proporcion_top
genero,5163,2,Masculino,2619,0.507263
jubilado,5163,2,No,4497,0.871005
en_pareja,5163,2,Si,2724,0.5276
dependientes,5163,2,No,3390,0.656595
servicio_telefonico,5163,2,Si,4653,0.90122
multiples_lineas,5163,3,No,2536,0.491187
servicio_internet,5163,3,DSL,1957,0.379043
seguridad_online,5163,3,No,2036,0.394344
respaldo_online,5163,3,Si,1902,0.36839
proteccion_dispositivo,5163,3,No,1883,0.36471


> Tenemos 5163 clientes que mantienen el servicio con la empresa

> La proporcion entre hombres y mujeres es igual en este grupo de clientes.

> 87% de los clientes no son adultos mayores.

> 52% estan en pareja.

> 65% no tienen dependientes a su cargo.

> Mas del 90% de los clientes cuentan con servicio telefonico.

> 49% cuentan con al menos 1 linea telefonica.

> 38% de los clientes cuentan con servicio de internet a traves de DSL.

> 39% no cuenta con seguridad online.

> 37% de los clientes cuentan con respaldo online.

> 36% no cuentan con proteccion de dispositivos

> 39% de los clientes no posee soporte tecnico efectivo.

> 36% si cuentan con servicio de streaming TV.

> 37% si cuentan con servicio de streaming de peliculas

> Solo el 43% de estos clientes prefieren el tipo de contrato es mes a mes.

> Mas del 53% reciben facturacion eletronica.

> 25.1% de los clientes prefieren el Cheque por correo como metodo de pago. 

### Resumen estadistico de las variables numericas de clientes que cancelan

In [ ]:
df_n_cancelan = df_exp[df_exp['cancelacion'] == 1]
df_n_cancelan = df_n_cancelan.describe()
df_n_cancelan.T

,count,mean,std,min,25%,50%,75%,max
cancelacion,1869.0,1.000000,0.000000,1.000000,1.00,1.000000,1.0000,1.000
meses_contrato,1869.0,17.979133,19.531123,1.000000,2.00,10.000000,29.0000,72.000
cuenta_mensual,1869.0,74.441332,24.666053,18.850000,56.15,79.650000,94.2000,118.350
cuentas_total,1869.0,1531.796094,1890.822994,18.850000,134.50,703.550000,2331.3000,8684.800
cuentas_diarias,1869.0,0.637061,0.808238,0.010601,0.09,0.249306,0.8175,3.415


> En total, hay 1869 clientes que cancelan el servicio con la empresa.

> Aunque en promedio los clientes duren 18 meses, la mayoria cancelan el servicio antes de cumplir 1 año (mediana = 10 meses). La alta dispersion (std = 19 meses) indica una gran variabilidad en el tiempo que toman los clientes antes de cancelar.

> El rango de la cuenta mensual va desde 18.85 a 118.35 con una media de 74.44. En teoria, para una distribucion normal, la media deberia estar cerca 68.85 lo que evidencia una ligera asimetria hacia la izquierda. Muchos de los clientes que cancelan pagan cuentas mensuales medianamente altas.

> La gran diferencia entre la media (1531.79) y la mediana (703.55) en la cuenta total, indica la presencia de outliers, es decir clientes que pagan mucho en cuentas totales. Un valor maximo de 8684.8 y una alta variabilidad (std=1890), reafirma la presencia de valores atipicos.

> La media (0.64) y la mediana (0.25) de la cuenta diaria evidencia una distribucion sesgada a la derecha por la presencia de outliers. Lo que quiere decir es que hay pocos clientes que gastan mucho cada dia.

### Resumen estadistico de las variables numericas de clientes que no cancelan

In [329]:
df_n_Nocancelan = df_exp[df_exp['cancelacion'] == 0]
df_n_Nocancelan = df_n_Nocancelan.describe()
df_n_Nocancelan.T

,count,mean,std,min,25%,50%,75%,max
cancelacion,5163.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
meses_contrato,5163.0,37.650010,24.076940,1.000000,15.000000,38.000000,61.000000,72.000
cuenta_mensual,5163.0,61.307408,31.094557,18.250000,25.100000,64.450000,88.475000,118.750
cuentas_total,5163.0,2555.344141,2329.456984,18.800000,577.825000,1683.600000,4264.125000,8672.450
cuentas_diarias,5163.0,0.160736,0.328574,0.008935,0.036449,0.055444,0.114517,3.195


> En total, 5163 clientes mantienen el servicio con la empresa.

> La mitad de los clientes tienen contratos mayores a 38 meses (mediana=38). Se tratan de clientes leales con mas de 3 años en la empresa. Comparados con los que cancelan, estos clientes duran el doble de tiempo.

> En promedio, estos clientes pagan cuotas mensuales de 61.31 mucho menos que los clientes que cancelan (74.44)

> En promedio los clientes que se mantienen en la empresa gastan mas en cuenta total (2555.34) debido a que llevan mas tiempo con la empresa. 50% de los clientes gastan menos de ese valor total.

> A pesar de tener mas tiempo, estos clientes gastan a diario en promedio menos (0.167) que los que cancelan (0.637). Con una mediana baja (0.055) se intuye que muchos tienen planes economicos o uso moderado del servicio.